In [3]:
import numpy as np
from PIL import Image
test_img = Image.open(r"smile16.png")
a = np.array(test_img)[:,:,0]

img = Image.fromarray(np.uint8(a), 'L')
img.show()

In [70]:
from qiskit import transpile
from qiskit.providers.aer import QasmSimulator
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit

num_qubits = 2

all_errors = []

for i in range(100):
    a1 = np.random.binomial(n=1, p=0.5, size=(2,2)) * 254 + 1
    
    a1 = a1.astype("float64").flatten()
    norm = np.sqrt(np.sum(a1 ** 2))
    a1 /= norm
    # print(np.linalg.norm(a1))

    simulator = QasmSimulator()
    qc = QuantumCircuit(QuantumRegister(num_qubits,'q'))
    qc.initialize(a1.flatten().tolist())
    qc += QFT(num_qubits=num_qubits)
    qc += QFT(num_qubits=num_qubits, inverse=True)

    qc.measure_all()
    qc.draw()

    # # compile the circuit down to low-level QASM instructions
    # # supported by the backend (not needed for simple circuits)
    compiled_circuit = transpile(qc, simulator)

    # Execute the circuit on the qasm simulator
    job = simulator.run(compiled_circuit, shots=100000)

    # Grab results from the job
    result = job.result()

    # Returns counts
    # print(result)
    counts = result.get_counts(compiled_circuit)
    # print(counts)
    a_q = np.zeros((2 ** num_qubits))
    for i in range(2 ** num_qubits):
        for key in counts:
            a_q[int(key, 2)] = counts[key]
    
    a_q = a_q.astype("float64").flatten()
    norm = np.sqrt(np.sum(a_q ** 2))
    a_q /= norm
    # a_q = a_q[::-1]
    # print(np.linalg.norm(a_q))
  
    a1 = a1 * 255 / np.max(a1)
    a_q = a_q * 255 / np.max(a_q)
#     print("og", a1)
#     print("new", a_q)
#     print(np.mean(a_q))
#     print("RMSE", np.sqrt(np.mean((a1-a_q)**2)))
    
    all_errors.append(np.sqrt(np.mean((a1-a_q)**2)))

print(np.mean(all_errors))
print(np.std(all_errors))

<ipython-input-70-b099177577e1>:21: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  qc += QFT(num_qubits=num_qubits)
<ipython-input-70-b099177577e1>:22: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  qc += QFT(num_qubits=num_qubits, inverse=True)


1.1121531309279726
0.5306663900119685
